In [ ]:
%matplotlib widget
from nicess.readouts import load_bifrost_readouts, load_bifrost_readout_times
from scipp import plot, array, group, groupby, scalar, collapse, concat
import scipp as sc
import numpy as np
import matplotlib.pyplot as pp

In [ ]:
wide_slit_file = 'mcstas_instrument/test/master_bifrost_20221028-113548_00000.h5'
narrow_slit_file = 'mcstas_instrument/test/master_bifrost_20221028-113444_00000.h5'
no_slit_fixed_two_theta = 'mcstas_instrument/test/master_bifrost_20221101-162531_00000.h5'
no_slit_fixed_two_theta2 = 'mcstas_instrument/test/master_bifrost_20221104-100819_00000.h5'

In [ ]:
rw = load_bifrost_readouts(wide_slit_file)
rn = load_bifrost_readouts(narrow_slit_file)

rw = load_bifrost_readouts(no_slit_fixed_two_theta2)

In [ ]:
rw_clocks = load_bifrost_readout_times(no_slit_fixed_two_theta2)
plot({k: rw_clocks[k][::100][:5000] for k in ('Event', 'Pulse')})
#plot(rw_clocks['tof'][::100][:5000])

In [ ]:
plot(rw_clocks['tof_high'][::100][:5000])

In [ ]:
plot({k: rw_clocks[k][::100][:5000] for k in ('event_high', 'pulse_high')})

In [ ]:
plot((rw_clocks['event_high'] - rw_clocks['pulse_high'])[::100][:5000])

In [ ]:
plot({k: rw_clocks[k][::100][:5000] for k in ('event_low', 'pulse_low')})

In [ ]:
rw.hist(ratio=300, time_of_flight=100).plot()

In [ ]:
#rn.hist(ratio=300, time_of_flight=100).plot()

In [ ]:
#plot({'wide': rw.bin(ratio=1000), 'narrow': 5*rn.bin(ratio=1000)})

In [ ]:
from nicess.bifrost import Tank

In [ ]:
bifrost = Tank.from_calibration()

In [ ]:
triplet = bifrost.channels[1].pairs[0].detector

In [ ]:
ratio_edges = triplet.a_minus_b_over_a_plus_b_edges().rename_dims({'tube': 'ratio'})
tube_index = array(values=[-1,0,-1,1,-1,2,-1], dims=['ratio'])

rwb = rw.bin(ratio=ratio_edges)
rwb.coords['tube'] = tube_index
import numpy as np
#rwb.coords['low'] = ratio_edges[:-1]
#rwb.coords['high'] = ratio_edges[1:]

# tube 1 has 'backwards' edges compared to 0 and 2;
edges = np.vstack((ratio_edges.values[:-1],ratio_edges.values[1:]))
edges[:,3] = edges[1, 3], edges[0, 3]
rwb.coords['edges'] = array(values=edges, dims=['edges','ratio'])

rwbg = group(rwb, 'tube')
if len(rwbg['tube', scalar(-1)].sizes) != 0:
    print("Inter-tube ranges should not have events, but do!")

tubes = array(values=[0,1,2], dims=['tube'])
tube_ratios = {f"tube {x:c}": rwbg['tube', x].copy() for x in tubes}
tp = {x: t.bin(ratio=1000) for x,t in tube_ratios.items()}
out = plot(tp)
#for l, h in zip(rwb.coords['low'].values, rwb.coords['high'].values):
#    out.ax.plot([l, l, h, h], [10000, 0, 0, 10000], '--k')
for l, h in rwb.coords['edges'].values.T:
    out.ax.plot([l, l, h, h], [10000, 0, 0, 10000], '--k')
out


In [ ]:
rwb

In [ ]:
rwb['tube', scalar(0)]

In [ ]:
#rwb['ratio', [1,3,5]]
a = rwb.copy()
del a.coords['ratio']
a['ratio', 1:6:2]

In [ ]:
# Verify that the intertubal bins contain no events
a['ratio', ::2].bins.size().sum()

In [ ]:
ccrwb = concat([v for v in collapse(rwb, 'tube').values() if v.values.sizes['event']], dim='tube')

In [ ]:
ccrwb

In [ ]:
#def monosub(ratio, low, high):
#    return (ratio - low) / (high - low)

def monosub2(ratio, edges):
    return (ratio - edges['edges', 0]) / (edges['edges', 1] - edges['edges', 0])

In [ ]:
ccrwb.transform_coords(['x'], graph={'x': monosub2})

In [ ]:
aofx = a['ratio', 1:6:2].transform_coords(['x'], graph={'x': monosub2})
aofx

In [ ]:
plot({k: v.bin(x=1) for k, v in collapse(aofx.group('tube'),'x').items()})

In [ ]:
aoft = aofx.group('tube')
#aoft = a.group('tube')['tube',1:].copy()
aoft

In [ ]:
aoft.coords['at'] = concat([t.at for t in triplet.tubes], dim='tube')
aoft.coords['to'] = concat([t.to for t in triplet.tubes], dim='tube')
aoft

In [ ]:
def event_position(at, to, x):
    return x * (to - at) + at

In [ ]:
# ccrwb had coordinates/attributes 'ratio', 'edges', 'at', and 'to'; and so could be transformed in one go
#q = ccrwb.transform_coords(['position'], graph={'x': monosub, 'position': event_position})
q = aoft.transform_coords(['position'], graph={'position': event_position})
q

In [ ]:
q['tube',0].data.values

In [ ]:
#q.bin(tube=1).data.values[0].plot(projection='3d', positions='position')

In [ ]:
secondary = bifrost.to_secondary()
print(f"{len(secondary.detectors)=}\n{len(secondary.analyzers)=}\n{[x for x in dir(secondary) if '__' not in x]}")

In [ ]:
secondary.analyzer_map['channel', 1]['pair', 0]

In [ ]:
secondary.detector_map['channel', 1]['pair', 0]

In [ ]:
secondary.continuous_final_distance(secondary.detector_map['channel', 1]['pair', 0]['tube', 0].value, 0.1)

In [ ]:
def l1():
    return scalar(160, unit='m')

def secondary_index(cassette, pair, tube):
    # Figure out how to do this robustly for all detectors
    #print(cassette)
    #idx = secondary.detector_map['channel', cassette]['pair', pair]['tube', tube]
    #return idx
    return tube + scalar(15)

def l2(secondary_index, x):
    return secondary.broadcast_continuous_final_distance(secondary_index, x)

def velocity(l1, l2, time_of_flight):
    return (l1 + l2) / time_of_flight

def energy(velocity):
    from scipp.constants import neutron_mass
    return (scalar(0.5) * neutron_mass * velocity * velocity).to(unit='meV')

def wavelength(velocity):
    from scipp.constants import Planck, neutron_mass
    return (Planck / (neutron_mass * velocity)).to(unit='angstrom')

def theta(secondary_index, x):
    return secondary.broadcast_continuous_theta(secondary_index, x)
#    from scipp import sqrt, dot, acos
#    sa = secondary.continuous_analyzer_vector(i, x)
#    sd = secondary.continuous_detector_vector(i, x)
#    ad = sd - sa
#    return acos(dot(sa, ad)/sqrt(dot(sa, sa))/sqrt(ad, ad))

def delta_a4(secondary_index, x):
    return secondary.broadcast_continuous_delta_a4(secondary_index, x).to(unit='degree')

def d_spacing(wavelength, theta):
    from scipp import sin
    return wavelength / (2 * sin(theta))

def theta_d(theta):
    return theta.to(unit='degree')

graph = {
    'l1': l1, 
    'secondary_index': secondary_index, 
    'l2': l2, 
    'velocity': velocity, 
    'wavelength': wavelength, 
    'theta': theta, 
    'd_spacing': d_spacing, 
    'energy': energy, 
    'theta_d': theta_d,
    'delta_a4': delta_a4
}

#sc.show_graph(graph)

In [ ]:
aofd = aofx.transform_coords(['delta_a4', 'theta_d', 'd_spacing', 'energy', 'tube', 'x'], graph=graph, keep_inputs=True, keep_intermediate=True)
aofd

In [ ]:
plot({k: v.bin(energy=1) for k, v in collapse(aofd,'tube').items()})

In [ ]:
collapse(aofd, 'tube')['x:0'].bin(wavelength=1)

In [ ]:
plot({k: v.hist(delta_a4=100, x=31) for k, v in collapse(aofd, 'tube').items()})

In [ ]:
biggraph = {'l1': l1, 'secondary_index': secondary_index, 'l2': l2, 'velocity': velocity, 'wavelength': wavelength, 'theta': theta, 'd_spacing': d_spacing, 'energy': energy, 'x': monosub2}
b = a['ratio', 1:6:2].transform_coords(['tube','x', 'energy'], graph=biggraph)

In [ ]:
b

In [ ]:
b['energy',0].values.hist(x=100, energy=100).plot()

In [ ]:
b['energy',1].values.hist(x=100, energy=100).plot()

In [ ]:
b['energy',2].values.hist(x=100, energy=100).plot()